In [1]:
global driveRoot
driveRoot = '/home/balu/MyFiles/ThesisReports/BTP/ARproject/VisionAR/AR_GL/trainedModels/mlModels/fRCNN'
dataRoot = '/home/balu/MyFiles/ThesisReports/BTP/ARproject/VisionAR/AR_GL/trainedModels'

In [ ]:
!pip install tensorflow_gpu==1.15

In [ ]:
!pip install numpy==1.19.5
!pip uninstall -y pycocotools
!pip install pycocotools --no-binary pycocotools

## Configs and Hyperparameters

Support a variety of models, you can find more pretrained model from [Tensorflow detection model zoo: COCO-trained models](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/detection_model_zoo.md#coco-trained-models), as well as their pipline config files in [object_detection/samples/configs/](https://github.com/tensorflow/models/tree/master/research/object_detection/samples/configs).

In [2]:
# If you forked the repo, you can replace the link.
repo_url = 'https://github.com/roboflow-ai/tensorflow-object-detection-faster-rcnn'

# Number of training steps - 1000 will train very quickly, but more steps will increase accuracy.
num_steps = 20000  # 200000 to improve

# Number of evaluation steps.
num_eval_steps = 50

MODELS_CONFIG = {
    'ssd_mobilenet_v2': {
        'model_name': 'ssd_mobilenet_v2_coco_2018_03_29',
        'pipeline_file': 'ssd_mobilenet_v2_coco.config',
        'batch_size': 12
    },
    'faster_rcnn_inception_v2': {
        'model_name': 'faster_rcnn_inception_v2_coco_2018_01_28',
        'pipeline_file': 'faster_rcnn_inception_v2_pets.config',
        'batch_size': 12
    },
    'rfcn_resnet101': {
        'model_name': 'rfcn_resnet101_coco_2018_01_28',
        'pipeline_file': 'rfcn_resnet101_pets.config',
        'batch_size': 8
    }
}

# Pick the model you want to use
# Select a model in `MODELS_CONFIG`.
selected_model = 'faster_rcnn_inception_v2'

# Name of the object detection model to use.
MODEL = MODELS_CONFIG[selected_model]['model_name']

# Name of the pipline file in tensorflow object detection API.
pipeline_file = MODELS_CONFIG[selected_model]['pipeline_file']

# Training batch size fits in Colabe's Tesla K80 GPU memory for selected model.
batch_size = MODELS_CONFIG[selected_model]['batch_size']

## Clone the `tensorflow-object-detection` repository or your fork.

In [ ]:
# import os

# # %cd /content

# repo_dir_path = os.path.abspath(os.path.join('.', os.path.basename(repo_url)))
# !git clone {repo_url}
# %cd {repo_dir_path}
# !git pull

## Install required packages

In [3]:
%cd {driveRoot}
# !git clone --quiet https://github.com/tensorflow/models.git

!sudo -S apt-get install -qq protobuf-compiler python-pil python-lxml python-tk < password.txt

!sudo -S pip install -q Cython contextlib2 pillow lxml matplotlib pycocotools  < password.txt

!sudo -S pip install tf_slim scipy < password.txt
# %cd {driveRoot}/models/research
%cd {dataRoot}/models/research
!sudo -S protoc object_detection/protos/*.proto --python_out=. < ../../password.txt

import os
# os.environ['PYTHONPATH'] += ':/content/models/research/:/content/models/research/slim/'
# os.environ['PYTHONPATH'] += ':/models/research/:/models/research/slim/'

%env PYTHONPATH = $dataRoot/models/research/
import os 
os.environ['PYTHONPATH'] += ":/home/balu/MyFiles/ThesisReports/BTP/ARproject/VisionAR/AR_GL/trainedModels/models/research/slim"

! echo $PYTHONPATH
# !sudo -S cp object_detection/packages/tf2/setup.py . < ../../password.txt
# !pip install .

!python object_detection/builders/model_builder_test.py

/home/balu/MyFiles/ThesisReports/BTP/ARproject/VisionAR/AR_GL/trainedModels/mlModels/fRCNN
[sudo] password for balu: [sudo] password for balu: [sudo] password for balu: Requirement already satisfied: tf_slim in /usr/local/lib/python3.8/dist-packages (1.1.0)
/home/balu/MyFiles/ThesisReports/BTP/ARproject/VisionAR/AR_GL/trainedModels/models/research
/bin/bash: ../../password.txt: No such file or directory
env: PYTHONPATH=/home/balu/MyFiles/ThesisReports/BTP/ARproject/VisionAR/AR_GL/trainedModels/models/research/
/home/balu/MyFiles/ThesisReports/BTP/ARproject/VisionAR/AR_GL/trainedModels/models/research/:/home/balu/MyFiles/ThesisReports/BTP/ARproject/VisionAR/AR_GL/trainedModels/models/research/slim


## Prepare `tfrecord` files

Roboflow automatically creates our TFRecord and label_map files that we need!

**Generating your own TFRecords the only step you need to change for your own custom dataset.**

Because we need one TFRecord file for our training data, and one TFRecord file for our test data, we'll create two separate datasets in Roboflow and generate one set of TFRecords for each.

To create a dataset in Roboflow and generate TFRecords, follow [this step-by-step guide](https://blog.roboflow.ai/getting-started-with-roboflow/).

In [4]:
%cd {dataRoot}/tensorflow-object-detection-faster-rcnn/data

/home/balu/MyFiles/ThesisReports/BTP/ARproject/VisionAR/AR_GL/trainedModels/tensorflow-object-detection-faster-rcnn/data


In [ ]:
# UPDATE THIS LINK - get our data from Roboflow
!curl -L https://app.roboflow.ai/ds/sgJjPlKOb7?key=KSBdLohPE5 > roboflow.zip; unzip roboflow.zip; rm roboflow.zip


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    68  100    68    0     0     42      0  0:00:01  0:00:01 --:--:--    42
100   883  100   883    0     0    315      0  0:00:02  0:00:02 --:--:--     0
100 40.8M  100 40.8M    0     0  8206k      0  0:00:05  0:00:05 --:--:-- 23.4M
Archive:  roboflow.zip
replace README.dataset.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: ^C


In [ ]:
%ls

FYI.txt  README.dataset.txt  README.roboflow.txt  test/  train/  valid/


In [ ]:
# check out what we have in train
%ls train

Gestures_label_map.pbtxt  Gestures.tfrecord


In [ ]:
# show what we have in test
# %ls test

In [5]:
# NOTE: Update these TFRecord names from "cells" and "cells_label_map" to your files!
test_record_fname = dataRoot +'/tensorflow-object-detection-faster-rcnn/data/test/Gestures.tfrecord'
# test_record_fname = dataRoot +'/tensorflow-object-detection-faster-rcnn/data/valid/Gestures.tfrecord'
train_record_fname = dataRoot +'/tensorflow-object-detection-faster-rcnn/data/train/Gestures.tfrecord'
label_map_pbtxt_fname = dataRoot +'/tensorflow-object-detection-faster-rcnn/data/train/Gestures_label_map.pbtxt'

## Download base model

In [6]:
%cd {dataRoot}/models/research

import os
import shutil
import glob
import urllib.request
import tarfile
MODEL_FILE = MODEL + '.tar.gz'
DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/'
DEST_DIR = driveRoot + '/pretrained_model'

# if not (os.path.exists(MODEL_FILE)):
#     urllib.request.urlretrieve(DOWNLOAD_BASE + MODEL_FILE, MODEL_FILE)

# tar = tarfile.open(MODEL_FILE)
# # tar.extractall()
# tar.close()

# os.remove(MODEL_FILE)
# if (os.path.exists(DEST_DIR)):
#     shutil.rmtree(DEST_DIR)
# os.rename(MODEL, DEST_DIR)

/home/balu/MyFiles/ThesisReports/BTP/ARproject/VisionAR/AR_GL/trainedModels/models/research


In [7]:
!echo {DEST_DIR}
!ls -alh {DEST_DIR}

/home/balu/MyFiles/ThesisReports/BTP/ARproject/VisionAR/AR_GL/trainedModels/mlModels/fRCNN/pretrained_model
total 111M
drwxrwxr-x 3 balu balu 4.0K Apr 19 11:11 .
drwxrwxr-x 5 balu balu 4.0K Apr 20 16:01 ..
-rw-rw-r-- 1 balu balu   77 Feb  1  2018 checkpoint
-rw-rw-r-- 1 balu balu  55M Feb  1  2018 frozen_inference_graph.pb
-rw-rw-r-- 1 balu balu  51M Feb  1  2018 model.ckpt.data-00000-of-00001
-rw-rw-r-- 1 balu balu  16K Feb  1  2018 model.ckpt.index
-rw-rw-r-- 1 balu balu 5.5M Feb  1  2018 model.ckpt.meta
-rw-rw-r-- 1 balu balu 3.2K Feb  1  2018 pipeline.config
drwxrwxr-x 2 balu balu 4.0K Apr 19 11:11 saved_model


In [8]:
fine_tune_checkpoint = os.path.join(DEST_DIR, "model.ckpt")
fine_tune_checkpoint

'/home/balu/MyFiles/ThesisReports/BTP/ARproject/VisionAR/AR_GL/trainedModels/mlModels/fRCNN/pretrained_model/model.ckpt'

## Configuring a Training Pipeline

In [9]:
import os
pipeline_fname = os.path.join(dataRoot +'/models/research/object_detection/samples/configs/', pipeline_file)
assert os.path.isfile(pipeline_fname), '`{}` not exist'.format(pipeline_fname)

In [10]:
def get_num_classes(pbtxt_fname):
    from object_detection.utils import label_map_util
    label_map = label_map_util.load_labelmap(pbtxt_fname)
    categories = label_map_util.convert_label_map_to_categories(
        label_map, max_num_classes=90, use_display_name=True)
    category_index = label_map_util.create_category_index(categories)
    return len(category_index.keys())

In [11]:
import re

num_classes = get_num_classes(label_map_pbtxt_fname)
with open(pipeline_fname) as f:
    s = f.read()
with open(pipeline_fname, 'w') as f:
    
    # fine_tune_checkpoint
    s = re.sub('fine_tune_checkpoint: ".*?"',
               'fine_tune_checkpoint: "{}"'.format(fine_tune_checkpoint), s)
    
    # tfrecord files train and test.
    s = re.sub(
        '(input_path: ".*?)(train.record)(.*?")', 'input_path: "{}"'.format(train_record_fname), s)
    s = re.sub(
        '(input_path: ".*?)(val.record)(.*?")', 'input_path: "{}"'.format(test_record_fname), s)
    s = re.sub(
        '(input_path: ".*?)(valid)(.*?")', 'input_path: "{}"'.format(test_record_fname), s)

    # label_map_path
    s = re.sub(
        'label_map_path: ".*?"', 'label_map_path: "{}"'.format(label_map_pbtxt_fname), s)

    # Set training batch_size.
    s = re.sub('batch_size: [0-9]+',
               'batch_size: {}'.format(batch_size), s)

    # Set training steps, num_steps
    s = re.sub('num_steps: [0-9]+',
               'num_steps: {}'.format(num_steps), s)
    
    # Set number of classes num_classes.
    s = re.sub('num_classes: [0-9]+',
               'num_classes: {}'.format(num_classes), s)
    f.write(s)

In [12]:
!cat {pipeline_fname}

# Faster R-CNN with Inception v2, configured for Oxford-IIIT Pets Dataset.
# Users should configure the fine_tune_checkpoint field in the train config as
# well as the label_map_path and input_path fields in the train_input_reader and
# eval_input_reader. Search for "PATH_TO_BE_CONFIGURED" to find the fields that
# should be configured.

model {
  faster_rcnn {
    num_classes: 7
    image_resizer {
      keep_aspect_ratio_resizer {
        min_dimension: 600
        max_dimension: 1024
      }
    }
    feature_extractor {
      type: 'faster_rcnn_inception_v2'
      first_stage_features_stride: 16
    }
    first_stage_anchor_generator {
      grid_anchor_generator {
        scales: [0.25, 0.5, 1.0, 2.0]
        aspect_ratios: [0.5, 1.0, 2.0]
        height_stride: 16
        width_stride: 16
      }
    }
    first_stage_box_predictor_conv_hyperparams {
      op: CONV
      regularizer {
        l2_regularizer {
          weight: 0.0
        }
      }
      initializer {
        tru

In [13]:
model_dir = driveRoot+'/training'
# Optionally remove content in output model directory to fresh start.
# !rm -rf {model_dir}
# os.makedirs(model_dir, exist_ok=True)

## Run Tensorboard(Optional)

In [ ]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip -o ngrok-stable-linux-amd64.zip

In [ ]:
LOG_DIR = model_dir
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)

In [ ]:
get_ipython().system_raw('./ngrok http 6006 &')

### Get Tensorboard link

In [ ]:
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

In [ ]:
!pip install tensorboard


In [ ]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [ ]:
%tensorboard --logdir {driveRoot}/training --host localhost --port 8089


## Train the model

In [ ]:
!pip install lvis

In [ ]:
!python {driveRoot}/models/research/object_detection/model_main.py \
    --pipeline_config_path={pipeline_fname} \
    --model_dir={model_dir} \
    --alsologtostderr \
    --num_train_steps={num_steps} \
    --num_eval_steps={num_eval_steps}

python: can't open file '/home/balu/MyFiles/ThesisReports/BTP/ARproject/VisionAR/AR_GL/trainedModels/fRCNN/models/research/object_detection/model_main.py': [Errno 2] No such file or directory


In [15]:
!python {dataRoot}/models/research/object_detection/model_main.py \
--pipeline_config_path={pipeline_fname} \
--model_dir={model_dir} \
--checkpoint_dir={model_dir} \
--run_once=True

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
!ls {model_dir}

checkpoint
eval_0
events.out.tfevents.1650218167.coalabserver
export
graph.pbtxt
model.ckpt-13225.data-00000-of-00001
model.ckpt-13225.index
model.ckpt-13225.meta
model.ckpt-17539.data-00000-of-00001
model.ckpt-17539.index
model.ckpt-17539.meta
model.ckpt-20000.data-00000-of-00001
model.ckpt-20000.index
model.ckpt-20000.meta
model.ckpt-4405.data-00000-of-00001
model.ckpt-4405.index
model.ckpt-4405.meta
model.ckpt-8735.data-00000-of-00001
model.ckpt-8735.index
model.ckpt-8735.meta


## Exporting a Trained Inference Graph
Once your training job is complete, you need to extract the newly trained inference graph, which will be later used to perform the object detection. This can be done as follows:

In [ ]:
import re
import numpy as np

output_directory = driveRoot + '/fine_tuned_model'

lst = os.listdir(model_dir)
lst = [l for l in lst if 'model.ckpt-' in l and '.meta' in l]
steps=np.array([int(re.findall('\d+', l)[0]) for l in lst])
last_model = lst[steps.argmax()].replace('.meta', '')

last_model_path = os.path.join(model_dir, last_model)
print(last_model_path)
!python {driveRoot}/models/research/object_detection/export_inference_graph.py \
    --input_type=image_tensor \
    --pipeline_config_path={pipeline_fname} \
    --output_directory={output_directory} \
    --trained_checkpoint_prefix={last_model_path}

/home/balu/MyFiles/ThesisReports/BTP/ARproject/VisionAR/AR_GL/trainedModels/mlModels/fRCNN/training/model.ckpt-20000
python: can't open file '/home/balu/MyFiles/ThesisReports/BTP/ARproject/VisionAR/AR_GL/trainedModels/mlModels/fRCNN/models/research/object_detection/export_inference_graph.py': [Errno 2] No such file or directory


In [ ]:
!ls {output_directory}

checkpoint			model.ckpt.index  saved_model
frozen_inference_graph.pb	model.ckpt.meta
model.ckpt.data-00000-of-00001	pipeline.config


## Download the model `.pb` file

In [ ]:
import os

pb_fname = os.path.join(os.path.abspath(output_directory), "frozen_inference_graph.pb")
assert os.path.isfile(pb_fname), '`{}` not exist'.format(pb_fname)

In [ ]:
!ls -alh {pb_fname}

-rw-rw-r-- 1 balu balu 51M Apr 17 07:13 /home/balu/MyFiles/ThesisReports/BTP/ARproject/VisionAR/AR_GL/trainedModels/mlModels/fRCNN/fine_tuned_model/frozen_inference_graph.pb


### Option1 : upload the `.pb` file to your Google Drive
Then download it from your Google Drive to local file system.

During this step, you will be prompted to enter the token.

In [ ]:
# Install the PyDrive wrapper & import libraries.
# This only needs to be done once in a notebook.
!pip install -U -q PyDrive
!pip install google-colab
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials


# Authenticate and create the PyDrive client.
# This only needs to be done once in a notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

fname = os.path.basename(pb_fname)
# Create & upload a text file.
uploaded = drive.CreateFile({'title': fname})
uploaded.SetContentFile(pb_fname)
uploaded.Upload()
print('Uploaded file with ID {}'.format(uploaded.get('id')))

### Option2 :  Download the `.pb` file directly to your local file system
This method may not be stable when downloading large files like the model `.pb` file. Try **option 1** instead if not working.

In [ ]:
from google.colab import files
files.download(pb_fname)

### OPTIONAL: Download the `label_map.pbtxt` file

In [ ]:
from google.colab import files
files.download(label_map_pbtxt_fname)

### OPTIONAL: Download the modified pipline file
If you plan to use OpenVINO toolkit to convert the `.pb` file to inference faster on Intel's hardware (CPU/GPU, Movidius, etc.)

In [ ]:
files.download(pipeline_fname)

In [ ]:
# !tar cfz fine_tuned_model.tar.gz fine_tuned_model
# from google.colab import files
# files.download('fine_tuned_model.tar.gz')

## Run inference test

To test on your own images, you need to upload raw test images to the `test` folder located inside `/data`.

Right now, this folder contains TFRecord files from Roboflow. We need the raw images.


#### Add test images to this notebook

We can download the exact same raw images that are in our Roboflow test split to our local computer by downloading the images in a different (non-TFRecord) format.

Go back to our [dataset](https://public.roboflow.ai/object-detection/bccd/1), click "Download," select "COCO JSON" as the format, and download to your local machine.

Unzip the downloaded file, and navigate to the `test` directory.
![folder](https://i.imgur.com/xkjxmKP.png)


Now, on the left-hand side in the colab notebook, select the folder icon.
![Colab folder](https://i.imgur.com/59v08qG.png)

Right-click on `test`, and select "Upload." Navigate to the files locally on your machine you just downloaded...and voila! You're set!


In [ ]:
# optionally, remove the TFRecord and cells_label_map.pbtxt from
# the test directory so it is only raw images
repo_dir_path = dataRoot+'/tensorflow-object-detection-faster-rcnn/'
%cd {repo_dir_path}
%cd data/test
%rm Gestures.tfrecord
%rm Gestures_label_map.pbtxt

/home/balu/MyFiles/ThesisReports/BTP/ARproject/VisionAR/AR_GL/trainedModels/tensorflow-object-detection-faster-rcnn
/home/balu/MyFiles/ThesisReports/BTP/ARproject/VisionAR/AR_GL/trainedModels/tensorflow-object-detection-faster-rcnn/data/test
rm: cannot remove 'Gestures.tfrecord': No such file or directory
rm: cannot remove 'Gestures_label_map.pbtxt': No such file or directory


In [ ]:
import os
import glob
from natsort import natsorted
# Path to frozen detection graph. This is the actual model that is used for the object detection.
PATH_TO_CKPT = pb_fname

# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = label_map_pbtxt_fname

# If you want to test the code with your images, just add images files to the PATH_TO_TEST_IMAGES_DIR.
PATH_TO_TEST_IMAGES_DIR =  os.path.join(repo_dir_path, "data/test")

TEST_IMAGE_PATHS = PATH_TO_TEST_IMAGES_DIR 

assert os.path.isfile(pb_fname)
assert os.path.isfile(PATH_TO_LABELS)
if len(TEST_IMAGE_PATHS) == 0:
  sample_img = 'https://storage.googleapis.com/roboflow-platform-transforms/Ly2DeBzbwsemGd2ReHk4BFxy8683/cf5ed147e4f2675fbabbc9b0db750ecf/transformed.jpg'
  import urllib.request
  urllib.request.urlretrieve(sample_img, 
                            PATH_TO_TEST_IMAGES_DIR + "/cell.jpg")

TEST_IMAGE_PATHS = glob.glob(os.path.join(PATH_TO_TEST_IMAGES_DIR, "*.*"))
assert len(TEST_IMAGE_PATHS) > 0, 'No image found in `{}`.'.format(PATH_TO_TEST_IMAGES_DIR)
TEST_IMAGE_PATHS = natsorted(TEST_IMAGE_PATHS)

In [ ]:
!pip install matplotlib
!pip install matplotlib-inline

In [ ]:
%cd {dataRoot}/models/research/object_detection

import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image

# This is needed since the notebook is stored in the object_detection folder.
# sys.path.append("..")
from object_detection.utils import ops as utils_ops


# This is needed to display the images.
%matplotlib inline


from object_detection.utils import label_map_util

from object_detection.utils import visualization_utils as vis_util

/home/balu/MyFiles/ThesisReports/BTP/ARproject/VisionAR/AR_GL/trainedModels/models/research/object_detection


In [ ]:
detection_graph = tf.Graph()
with detection_graph.as_default():
    od_graph_def = tf.GraphDef()
    with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')


label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(
    label_map, max_num_classes=num_classes, use_display_name=True)
category_index = label_map_util.create_category_index(categories)


def load_image_into_numpy_array(image):
    (im_width, im_height) = image.size
    return np.array(image.getdata()).reshape(
        (im_height, im_width, 3)).astype(np.uint8)

# Size, in inches, of the output images.
IMAGE_SIZE = (12, 8)


def run_inference_for_single_image(image, graph):
    with graph.as_default():
        with tf.Session() as sess:
            # Get handles to input and output tensors
            ops = tf.get_default_graph().get_operations()
            all_tensor_names = {
                output.name for op in ops for output in op.outputs}
            tensor_dict = {}
            for key in [
                'num_detections', 'detection_boxes', 'detection_scores',
                'detection_classes', 'detection_masks'
            ]:
                tensor_name = key + ':0'
                if tensor_name in all_tensor_names:
                    tensor_dict[key] = tf.get_default_graph().get_tensor_by_name(
                        tensor_name)
            if 'detection_masks' in tensor_dict:
                # The following processing is only for single image
                detection_boxes = tf.squeeze(
                    tensor_dict['detection_boxes'], [0])
                detection_masks = tf.squeeze(
                    tensor_dict['detection_masks'], [0])
                # Reframe is required to translate mask from box coordinates to image coordinates and fit the image size.
                real_num_detection = tf.cast(
                    tensor_dict['num_detections'][0], tf.int32)
                detection_boxes = tf.slice(detection_boxes, [0, 0], [
                                           real_num_detection, -1])
                detection_masks = tf.slice(detection_masks, [0, 0, 0], [
                                           real_num_detection, -1, -1])
                detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
                    detection_masks, detection_boxes, image.shape[0], image.shape[1])
                detection_masks_reframed = tf.cast(
                    tf.greater(detection_masks_reframed, 0.5), tf.uint8)
                # Follow the convention by adding back the batch dimension
                tensor_dict['detection_masks'] = tf.expand_dims(
                    detection_masks_reframed, 0)
            image_tensor = tf.get_default_graph().get_tensor_by_name('image_tensor:0')

            # Run inference
            output_dict = sess.run(tensor_dict,
                                   feed_dict={image_tensor: np.expand_dims(image, 0)})

            # all outputs are float32 numpy arrays, so convert types as appropriate
            output_dict['num_detections'] = int(
                output_dict['num_detections'][0])
            output_dict['detection_classes'] = output_dict[
                'detection_classes'][0].astype(np.uint8)
            output_dict['detection_boxes'] = output_dict['detection_boxes'][0]
            output_dict['detection_scores'] = output_dict['detection_scores'][0]
            if 'detection_masks' in output_dict:
                output_dict['detection_masks'] = output_dict['detection_masks'][0]
    return output_dict

In [ ]:
# Output images not showing? Run this cell again, and try the cell above
# This is needed to display the images.
%matplotlib inline

In [ ]:

count = 0
for image_path in TEST_IMAGE_PATHS:
    # if count == 5:
    #   break
    image = Image.open(image_path)
    # the array based representation of the image will be used later in order to prepare the
    # result image with boxes and labels on it.
    image_np = load_image_into_numpy_array(image)
    # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
    image_np_expanded = np.expand_dims(image_np, axis=0)
    # Actual detection.
    output_dict = run_inference_for_single_image(image_np, detection_graph)
    # Visualization of the results of a detection.
    vis_util.visualize_boxes_and_labels_on_image_array(
        image_np,
        output_dict['detection_boxes'],
        output_dict['detection_classes'],
        output_dict['detection_scores'],
        category_index,
        instance_masks=output_dict.get('detection_masks'),
        use_normalized_coordinates=True,
        line_thickness=8)
    plt.figure(figsize=IMAGE_SIZE)
    plt.imshow(image_np)

Output hidden; open in https://colab.research.google.com to view.